In [8]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def read(path):
	for root, dn, fns in os.walk(path):
		for fn in fns:
			path = os.path.join(root, fn)
			
			inBody = False
			lines = []
			f = io.open(path, 'r', encoding='latin1')
			for l in f:
				if inBody:
					lines.append(l)
				elif l == '\n':
					inBody = True
			f.close()
			msg = '\n'.join(lines)
			yield path, msg

def dffromdir(path, clas):
	rows = []
	index = []
	for fn, msg in read(path):
		rows.append({'message': msg, 'class' : clas})
		index.append(fn)
	return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})
data = data.append(dffromdir('emails/spam', 'spam'))
data = data.append(dffromdir('emails/ham', 'ham'))  

data.head()

vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], dtype='<U4')